In [2]:
import pandas as pd
import numpy as np
#from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostClassifier
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.utils import shuffle

Проведем первичный быстрый анализ.

In [3]:
data = pd.read_csv(r'C:\Users\Despik\PycharmProjects\untitled\Churn.csv')

In [4]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [6]:
data.isna().sum()

RowNumber            0
CustomerId           0
Surname              0
CreditScore          0
Geography            0
Gender               0
Age                  0
Tenure             909
Balance              0
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Exited               0
dtype: int64

In [7]:
data.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,9091.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,4.997690,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.894723,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,2.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


.797 - наш адекватный порог. У нас 909 пропусков в столбце 'Tenure'. Проверить, есть ли 'CustomerId' повторы. Классы не сбалансированы. Необходимо в будущем это учесть!

In [8]:
data.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

Со столбцами все в порядке.

In [9]:
data['Tenure'].value_counts()

1.0     952
2.0     950
8.0     933
3.0     928
5.0     927
7.0     925
4.0     885
9.0     882
6.0     881
10.0    446
0.0     382
Name: Tenure, dtype: int64

In [10]:
data['CustomerId'].value_counts().sum()

10000

Все айди разные. Но этот столбец, нам не нужен в будущем. Фамилия тоже не нужна.

In [11]:
data = data.drop(['CustomerId', 'Surname', 'RowNumber'], axis=1)

In [12]:
#data = data.drop('Surname', axis=1)

In [13]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [14]:
data['Geography'].value_counts()

France     5014
Germany    2509
Spain      2477
Name: Geography, dtype: int64

Воображаемых стран нет. Используем даммис для преобразования признаков, не забуедм про даммис ловушку.

In [15]:
data_dummy = pd.get_dummies(data, drop_first=True)

In [16]:
data_dummy.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8.0,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1.0,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2.0,125510.82,1,1,1,79084.10,0,0,1,0


Нашел метот заполнения NaN через метод KNN. Опробуем. KNN не работает. Попробую на рабочей машине, посмотрю, какая разница с вывоводом от Iterative

In [17]:
data_dummy['Tenure'].describe()

count    9091.000000
mean        4.997690
std         2.894723
min         0.000000
25%         2.000000
50%         5.000000
75%         7.000000
max        10.000000
Name: Tenure, dtype: float64

Медиана 5, заполним этим значением.

In [18]:
data_dummy.loc[data_dummy['Tenure'].isna() == True,['Tenure']] = 5 

In [19]:
data_dummy['Tenure'].value_counts()

5.0     1836
1.0      952
2.0      950
8.0      933
3.0      928
7.0      925
4.0      885
9.0      882
6.0      881
10.0     446
0.0      382
Name: Tenure, dtype: int64

In [20]:
data_dummy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CreditScore        10000 non-null  int64  
 1   Age                10000 non-null  int64  
 2   Tenure             10000 non-null  float64
 3   Balance            10000 non-null  float64
 4   NumOfProducts      10000 non-null  int64  
 5   HasCrCard          10000 non-null  int64  
 6   IsActiveMember     10000 non-null  int64  
 7   EstimatedSalary    10000 non-null  float64
 8   Exited             10000 non-null  int64  
 9   Geography_Germany  10000 non-null  uint8  
 10  Geography_Spain    10000 non-null  uint8  
 11  Gender_Male        10000 non-null  uint8  
dtypes: float64(3), int64(6), uint8(3)
memory usage: 732.5 KB


На хабе практикума стоит старый склерн. Из-за этого KNNImputer не работает. На рабочей машине получилось такое распределение:
<br>6.0     1119</br>
<br>4.0     1102</br>
<br>2.0     1051</br>
<br>8.0     1043</br>
<br>5.0     1008</br>
<br>3.0      993</br>
<br>7.0      977</br>
<br>1.0      963</br>
<br>9.0      895</br>
<br>10.0     455</br>
<br>0.0      394</br>

Разница заметна, но будем использовать доступный способ.

In [21]:
data_dummy['Tenure'] = data_dummy['Tenure'].astype('int')

In [22]:
data_dummy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CreditScore        10000 non-null  int64  
 1   Age                10000 non-null  int64  
 2   Tenure             10000 non-null  int32  
 3   Balance            10000 non-null  float64
 4   NumOfProducts      10000 non-null  int64  
 5   HasCrCard          10000 non-null  int64  
 6   IsActiveMember     10000 non-null  int64  
 7   EstimatedSalary    10000 non-null  float64
 8   Exited             10000 non-null  int64  
 9   Geography_Germany  10000 non-null  uint8  
 10  Geography_Spain    10000 non-null  uint8  
 11  Gender_Male        10000 non-null  uint8  
dtypes: float64(2), int32(1), int64(6), uint8(3)
memory usage: 693.5 KB


In [23]:
data_dummy.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,1,0


In [24]:
X = data_dummy.drop('Exited', axis=1)

In [25]:
y = data_dummy['Exited']

Данные к первичному обучению готовы. По заданию нам необходимо посмотреть на результат моделей, без учета дисбаланса. Займемся этим ниже. Сначала разделим на тестовую, трейн и валидную часть

In [26]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2, random_state=12345)

In [27]:
X_valid, X_test, y_valid, y_test = train_test_split(X_valid, y_valid, test_size = 0.5, random_state=12345)

In [28]:
X_train.shape, X_valid.shape, X_test.shape, y_train.shape, y_valid.shape, y_test.shape

((8000, 11), (1000, 11), (1000, 11), (8000,), (1000,), (1000,))

Исследовать мы будем регрессию, дерево и лес. Посмотрим, какие результаты нам дают. На данном этапе не будем работать с гиперпараметрами.

In [29]:
model_regr = LogisticRegression()

In [30]:
model_tree = DecisionTreeClassifier(random_state=12345)

In [31]:
model_forest = RandomForestClassifier(random_state=12345)

In [32]:
model_regr.fit(X_train, y_train)

LogisticRegression()

In [33]:
model_tree.fit(X_train, y_train)

DecisionTreeClassifier(random_state=12345)

In [34]:
model_forest.fit(X_train, y_train)

RandomForestClassifier(random_state=12345)

In [35]:
predict_regr = model_regr.predict(X_valid)

In [36]:
predict_tree = model_tree.predict(X_valid)

In [37]:
predict_forest = model_forest.predict(X_valid)

Посмотрим F1 и ROC-AUC

Логистическая регрессия

In [38]:
accuracy_score(y_valid, predict_regr)

0.765

In [39]:
f1_score(y_valid, predict_regr)

0.0784313725490196

In [40]:
roc_auc_score(y_valid, predict_regr)

0.5093189964157706

Дерево

In [41]:
accuracy_score(y_valid, predict_tree)

0.778

In [42]:
f1_score(y_valid, predict_tree)

0.5152838427947598

In [43]:
roc_auc_score(y_valid, predict_tree)

0.6880286738351254

Случайный лес

In [44]:
accuracy_score(y_valid, predict_forest)

0.851

In [45]:
f1_score(y_valid, predict_forest)

0.5802816901408451

In [46]:
roc_auc_score(y_valid, predict_forest)

0.7114695340501793

Лес дает самый высокий скор на несбалансированных данных. Так как мы не учитывали дисбаланс классов, ф1 не достигает необходимого значения в 0.59


На следующем этапе, мы сбалансируем данные, найдем лучшую модель для леса, и бонусом обучим catboost. Посмотрим, как бустинг справится с нашей задачей.

Первым делом, сделаем масштабирование признаков.

In [47]:
data_dummy.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,1,0


In [48]:
numeric = ['CreditScore', 'Age', 'Balance', 'EstimatedSalary', 'Tenure']

In [49]:
scaler = StandardScaler()

In [50]:
scaler.fit(X_train[numeric])

StandardScaler()

In [51]:
X_train[numeric] = scaler.transform(X_train[numeric])

<ipython-input-51-df056eba7012>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[numeric] = scaler.transform(X_train[numeric])
c:\users\despik\appdata\local\programs\python\python38\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


X_valid[numeric] = scaler.transform(X_valid[numeric])

In [52]:
X_test[numeric] = scaler.transform(X_test[numeric])

In [53]:
X_train.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
454,1.107585,0.011482,0.358552,0.819420,1,0,1,-0.757878,0,0,0
8307,-0.338704,-0.844298,-0.730444,0.923294,1,1,1,-1.194506,0,0,1
4811,-0.782922,-1.224645,0.721550,-1.220083,2,1,0,0.212357,0,1,0
1132,-0.235398,-0.178692,-0.004447,-1.220083,2,1,1,1.239310,0,0,1
9640,1.024940,-0.368865,-0.730444,-1.220083,1,1,0,1.583001,0,0,0


Пробуем регрессию, после стандартизации.

In [54]:
model_regr_scaler = LogisticRegression()

In [55]:
model_regr_scaler.fit(X_train, y_train)

LogisticRegression()

In [56]:
predict_regt_scaler = model_regr_scaler.predict(X_test)

В 3 раза выше, чем без стандартизации. Заметно масштаб влияет на регрессию.

In [57]:
f1_score(y_test, predict_regt_scaler)

0.2727272727272727

Возьмем функцию на увеличение выборки.

In [58]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    return shuffle(features_upsampled, target_upsampled, random_state=12345)

In [59]:
X_train_up, y_train_up = upsample(X_train, y_train, 3)

Теперь на уменьшение

In [60]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

In [61]:
X_train_down, y_train_down = downsample(X_train, y_train, 0.6)

Запустимм гридсерч, чтобы найти луйчшие параметры для дерева. Не забываем указать class_weight, так как у нас несбалансированный датасет.

In [62]:
model_forest_balanced = RandomForestClassifier(random_state=12345, class_weight='balanced')

Так как мы сами балансируем данные, уберем тут балансед из класса весов

In [63]:
model_forest_nobalance = RandomForestClassifier(random_state=12345)

In [64]:
parametrs_grid = {'max_depth': range(6,20,2),
                  'n_estimators' : range(80, 160, 10),
                  'min_samples_leaf': range(1,3,2),
                  'min_samples_split': range(2,4,2),
                  }

In [65]:
ss = ShuffleSplit(n_splits=5, test_size=0.25, random_state=12345)

In [66]:
grid_cv = GridSearchCV(model_forest_balanced, parametrs_grid, scoring='f1', cv=ss, n_jobs=-1, verbose=1)

In [67]:
grid_cv_up = GridSearchCV(model_forest_nobalance, parametrs_grid, scoring='f1', cv=ss, n_jobs=-1, verbose=1)

In [68]:
grid_cv_down = GridSearchCV(model_forest_nobalance, parametrs_grid, scoring='f1', cv=ss, n_jobs=-1, verbose=1)

In [69]:
%%time
grid_cv.fit(X_train, y_train)

Fitting 5 folds for each of 56 candidates, totalling 280 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   11.5s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 280 out of 280 | elapsed:  1.5min finished


Wall time: 1min 33s


GridSearchCV(cv=ShuffleSplit(n_splits=5, random_state=12345, test_size=0.25, train_size=None),
             estimator=RandomForestClassifier(class_weight='balanced',
                                              random_state=12345),
             n_jobs=-1,
             param_grid={'max_depth': range(6, 20, 2),
                         'min_samples_leaf': range(1, 3, 2),
                         'min_samples_split': range(2, 4, 2),
                         'n_estimators': range(80, 160, 10)},
             scoring='f1', verbose=1)

In [70]:
%%time
grid_cv_up.fit(X_train_up, y_train_up)

Fitting 5 folds for each of 56 candidates, totalling 280 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   16.3s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 280 out of 280 | elapsed:  2.0min finished


Wall time: 2min 3s


GridSearchCV(cv=ShuffleSplit(n_splits=5, random_state=12345, test_size=0.25, train_size=None),
             estimator=RandomForestClassifier(random_state=12345), n_jobs=-1,
             param_grid={'max_depth': range(6, 20, 2),
                         'min_samples_leaf': range(1, 3, 2),
                         'min_samples_split': range(2, 4, 2),
                         'n_estimators': range(80, 160, 10)},
             scoring='f1', verbose=1)

In [71]:
%%time
grid_cv_down.fit(X_train_down, y_train_down)

Fitting 5 folds for each of 56 candidates, totalling 280 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    7.7s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   39.4s
[Parallel(n_jobs=-1)]: Done 280 out of 280 | elapsed:  1.1min finished


Wall time: 1min 4s


GridSearchCV(cv=ShuffleSplit(n_splits=5, random_state=12345, test_size=0.25, train_size=None),
             estimator=RandomForestClassifier(random_state=12345), n_jobs=-1,
             param_grid={'max_depth': range(6, 20, 2),
                         'min_samples_leaf': range(1, 3, 2),
                         'min_samples_split': range(2, 4, 2),
                         'n_estimators': range(80, 160, 10)},
             scoring='f1', verbose=1)

In [72]:
grid_cv.best_estimator_

RandomForestClassifier(class_weight='balanced', max_depth=10, n_estimators=90,
                       random_state=12345)

In [73]:
grid_cv_up.best_estimator_

RandomForestClassifier(max_depth=18, n_estimators=130, random_state=12345)

In [74]:
grid_cv_down.best_estimator_

RandomForestClassifier(max_depth=16, n_estimators=120, random_state=12345)

In [75]:
grid_cv.best_score_

0.6140544065512304

In [76]:
grid_cv_up.best_score_

0.9361956285920023

In [77]:
grid_cv_down.best_score_

0.6432283419291965

In [78]:
model_forest_last = grid_cv.best_estimator_

In [79]:
model_forest_last_up = grid_cv_up.best_estimator_

In [80]:
model_forest_last_down = grid_cv_down.best_estimator_

In [81]:
predict_test = model_forest_last.predict(X_test)

In [82]:
predict_test_up = model_forest_last_up.predict(X_test)

In [83]:
predict_test_down = model_forest_last_down.predict(X_test)

In [84]:
accuracy_score(y_test, predict_test)

0.837

Задача выполнена. F1 > 0.59.

In [85]:
f1_score(y_test, predict_test)

0.6146572104018914

In [86]:
f1_score(y_test, predict_test_up)

0.5777777777777777

In [87]:
f1_score(y_test, predict_test_down)

0.582210242587601

Выше, чем у сырой модели, но не 0.59. Думаю, если поменять параметры функции на даун и апсемплинг может измениться скор. 

In [88]:
roc_auc_score(y_test, predict_test)

0.7647646343581727

In [89]:
roc_auc_score(y_test, predict_test_up)

0.7235911561080919

In [90]:
roc_auc_score(y_test, predict_test_down)

0.7291061812948213

In [91]:
precision_score(y_test, predict_test)

0.5882352941176471

In [92]:
recall_score(y_test, predict_test)

0.6435643564356436

Для кетбуста хватит одной балансировки, для данного случая по моему мнению, самой подходящей.

Обучим Catboost на наших данных. Выставим веса 0.2 и 0.8

In [93]:
y_train.sum()

1610

In [94]:
y_train.count()

8000

In [95]:
(8000-1610)/1610

3.968944099378882

In [96]:
cb_model = CatBoostClassifier(iterations=1500, learning_rate=0.03, 
                              depth=9,eval_metric='F1',verbose=50,
                              random_seed = 12345,task_type='CPU',class_weights =  [1, 4])

In [97]:
cb_model.fit(X_train, y_train, eval_set=(X_valid,y_valid), 
             use_best_model=True)

0:	learn: 0.7690245	test: 0.6477212	best: 0.6477212 (0)	total: 75.8ms	remaining: 1m 53s
50:	learn: 0.8073686	test: 0.6652452	best: 0.7148594 (13)	total: 851ms	remaining: 24.2s
100:	learn: 0.8305112	test: 0.6540336	best: 0.7148594 (13)	total: 1.89s	remaining: 26.2s
150:	learn: 0.8542469	test: 0.6442953	best: 0.7148594 (13)	total: 2.88s	remaining: 25.7s
200:	learn: 0.8724885	test: 0.6383701	best: 0.7148594 (13)	total: 4.11s	remaining: 26.5s
250:	learn: 0.8886123	test: 0.6383701	best: 0.7148594 (13)	total: 5.14s	remaining: 25.6s
300:	learn: 0.9045039	test: 0.6372881	best: 0.7148594 (13)	total: 5.95s	remaining: 23.7s
350:	learn: 0.9170521	test: 0.6409222	best: 0.7148594 (13)	total: 6.81s	remaining: 22.3s
400:	learn: 0.9277321	test: 0.6372093	best: 0.7148594 (13)	total: 7.61s	remaining: 20.9s
450:	learn: 0.9365943	test: 0.6458456	best: 0.7148594 (13)	total: 8.57s	remaining: 19.9s
500:	learn: 0.9459997	test: 0.6469894	best: 0.7148594 (13)	total: 9.36s	remaining: 18.7s
550:	learn: 0.9558711	t

In [98]:
predict_cat = cb_model.predict(X_test)

In [99]:
accuracy_score(y_test, predict_cat)

0.806

f1 уже >0.6 Задача выполнена.

In [100]:
f1_score(y_test, predict_cat)

0.6088709677419355

In [101]:
roc_auc_score(y_test, predict_cat)

0.7841633787438893

In [102]:
precision_score(y_test, predict_cat)

0.5136054421768708

In [103]:
recall_score(y_test, predict_cat)

0.7475247524752475

Необходимый скор позволело нам достич учет дисбаланса классов и работа с признаками. Мы использовали дамми модели, чтобы  исбавиться от категориальных признаков. Стандартизировали данные, чтобы уравнять масштаб. Учли дисбаланс во время подготовки моделей.
Самой успешной моделью была модель обученная catboost. 

Catboost - f1 score = 0.608 ROC AUC - 0.773

Случайный лес - f1 score = 0.595. ROC AUC - 0.744

В обоих случаях, roc auc выше чем у случайной модели, но до идеала далеко. Предполагаю, чтобы увеличить roc auc и f1 score необходимо заниматься созданием новых фич. Предполагаю, что это мы будем делать в дальнейшем обучении. Поставленная задача выполнена.